In [ ]:
import imageio
import glob
import re
from random import sample, seed
import os
import matplotlib.pyplot as plt
import numpy as np
from math import *
import pptk
import sys
import cv2
import os
import math
from os import walk
import re as reg

In [ ]:
seed(20)
if os.name == 'nt': # Windows
    system_win = 1
else:
    system_win = 0

In [ ]:
def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0: 
        return v
    return v / norm

In [ ]:
# This function will read a series of PNG file and convert its content to a 3d  array containing the volume, 
# Param: filePath: the file path of the PNG files. Each file should be named as 1.png, 2.png, 3.png ... etc. All the png file should 
#                  be ordered by the their topological order from their original dicom file
#        padding: If set to an interger x, then each png image will be copied x times and these slices will all be added
#               to the volume. This is used to pad between the slices.
# Return: a 3d numpy array representing the volume containing the stack of the png images and their copies.

def ReadPointFromPNG(filepath, padding:int):
    print("---------------")
    print("Begin reading points data from PNG files")
    path_list = [im_path for im_path in glob.glob(filepath)]
    
    if system_win:
        path_list_parsed = [re.split('\\\\|\.', path) for path in path_list]
    else:
        path_list_parsed = [re.split('/|\.', path) for path in path_list]
    path_list_parsed_valid = [x for x in path_list_parsed if x[-1] == 'png']
    path_list_parsed_valid = sorted(path_list_parsed_valid, key=lambda x:int(x[-2]))
    
    print("There are", len(path_list_parsed_valid),"PNG files, now concatinate them into a single 3D array")
    imageData = []
    
    for path in path_list_parsed_valid:
        s = ""
        if system_win:
            s = "\\"
        else:
            s = "/"
        s = s.join(path)
        s = s[:-4] + '.png'
        image = imageio.imread(s)
        
        for i in range(padding):
            imageData.append(image)
            
    imageData = np.asarray(imageData)
    imageData = np.rot90(imageData, 1, (0, 2))
    imageData = np.rot90(imageData, 1, (1, 0))
    
    print("Done!")
    return imageData

In [ ]:
def ProjectVectorOnToPlane(original_v, plane_normal):
    original_v = np.array(original_v)
    plane_normal = np.array(plane_normal)
    
    v_project_to_normal = np.dot(original_v, plane_normal)
    v_project_to_normal = v_project_to_normal / (np.dot(plane_normal,plane_normal))
    v_project_to_normal = np.dot(v_project_to_normal, plane_normal)
    
    v_project_to_plane = original_v - v_project_to_normal
    
    return normalize(v_project_to_plane)

In [ ]:
# Given a 3d volume, a plane's parameter (a,b,c,d as in ax + by + cz + d = 0) and the center point coordinate (must be on
# the plane specified), return the image correspond to the plane, centered at the certer point's location
# Param: data_volume: a 3d numpy array containing the volume data
#        planeParam: parameters of a plane (a,b,c,d as in ax + by + cz + d = 0)
#        centerPoint: the 3d coordinate of a point. The point has to be on the plane. The result image will be centered
#                     by this point. 
#        range: how large the result image will be. The larger the image, the slower the computation

def getTangentPlane (data_volume, planeParam, centerPoint, resultRangeHalf: int = 100):
    
    a, b, c, d = planeParam
    
    if(a * centerPoint[0] + b * centerPoint[1] + c * centerPoint[2] + d != 0):
        print("error: center point is not on the plane!")
        return
    
    data_volume = np.asarray(data_volume)
    bounds = data_volume.shape

    if( a == 0 and b !=0 and c != 0):
        u = np.array([0, 0, -d/c]) - np.array([0, -d/b , 0])
        u = u / np.linalg.norm(u)
    elif( b == 0 and a !=0 and c != 0):
        u = np.array([0, 0, -d/c]) - np.array([-d/a, 0, 0])
        u = u / np.linalg.norm(u)
    elif( c == 0 and a !=0 and b != 0):
        u = np.array([0, -d/b, 0]) - np.array([-d/a, 0, 0])
        u = u / np.linalg.norm(u)
    elif(a == 0 and b == 0):
        u = np.array([1, 0, 0])
    elif(a == 0 and c == 0):
        u = np.array([0, 0, 1])
    elif(b == 0 and c == 0):
        u = np.array([0, 0, 1])
    elif(a == 0 or b == 0 or c == 0):
        print("plane parameter error! a =", a, "b =", b, "c =", c, "d =", d)
    else:
        u = np.array([0, 0, -d/c]) - np.array([1, 1, (-d - a - b)/c])
        u = u / np.linalg.norm(u)
         
    normal = np.array([a, b, c])
    normal = normal / np.linalg.norm(normal)
    v = np.cross(u, normal)
    v = v / np.linalg.norm(v)

#    planeImg = np.zeros((halfDiagnalLength*2, halfDiagnalLength*2))
    
#     # filling the 2d Image, this part can be further optimized by using matrix
#     for x in range(-halfDiagnalLength, halfDiagnalLength):
#         for y in range(-halfDiagnalLength, halfDiagnalLength):
#             vectorElem1 = x*u
#             vectorElem2 = y*v
#             volumeCorFloat = vectorElem1 + vectorElem2 + centerPoint
#             volumeCor = np.array([int(round(volumeCorFloat[0])), int(round(volumeCorFloat[1])), int(round(volumeCorFloat[2]))])
#             #Check boundary
#             if(volumeCor[0] > 0 and volumeCor[0] < bounds[0] and volumeCor[1] > 0 and volumeCor[1] < bounds[1] and\
#               volumeCor[2] > 0 and volumeCor[2] < bounds[2]):
                
#                 volxelVal = data_volume[volumeCor[0]][volumeCor[1]][volumeCor[2]]
#                 planeImg[x + halfDiagnalLength][y + halfDiagnalLength] = volxelVal
    
    xyCorMatrix = [[x,y] for x in range(-resultRangeHalf, resultRangeHalf) \
                   for y in range(-resultRangeHalf, resultRangeHalf)]
    xyCorMatrix = np.array(xyCorMatrix)
    uvMat = np.array([u, v])
    dotMat = np.dot(xyCorMatrix, uvMat)
    dotMat += np.array(centerPoint)
    dotMat = np.round(dotMat).astype(int)    
    indexMat = np.transpose(dotMat)
    
    invalid = np.where((indexMat[0] <= 0) | (indexMat[0] >= bounds[0]) | \
                       (indexMat[1] <= 0) | (indexMat[1] >= bounds[1]) | \
                       (indexMat[2] <= 0) | (indexMat[2] >= bounds[2]))[0].tolist()
    
    indexMat[:, invalid] -= indexMat[:, invalid]
    
    rawImage = data_volume[indexMat[0], indexMat[1], indexMat[2]]
    planeImg = np.reshape(rawImage, (resultRangeHalf*2, resultRangeHalf*2))
    
    return planeImg

In [ ]:
# Given a 3d volume, a plane's parameter (a,b,c,d as in ax + by + cz + d = 0) and the center point coordinate (must be on
# the plane specified), return the image correspond to the plane, centered at the certer point's location
# Param: data_volume: a 3d numpy array containing the volume data
#        planeParam: parameters of a plane (a,b,c,d as in ax + by + cz + d = 0)
#        centerPoint: the 3d coordinate of a point. The point has to be on the plane. The result image will be centered
#                     by this point. 
#        range: how large the result image will be. The larger the image, the slower the computation

def getSmoothTangentPlane (data_volume, planeParam, centerPoint, prev_u: list = [0,0,0], first_call: bool = False, resultRangeHalf: int = 100):
    
    a, b, c, d = planeParam
    
    if(a * centerPoint[0] + b * centerPoint[1] + c * centerPoint[2] + d != 0):
        print("error: center point is not on the plane!")
        return
    
    data_volume = np.asarray(data_volume)
    bounds = data_volume.shape
    
    u = []
    v = []
    normal = np.array([a, b, c])
    normal = normalize(normal)
    
    if first_call:
        if( a == 0 and b !=0 and c != 0):
            u = np.array([0, 0, -d/c]) - np.array([0, -d/b , 0])
            u = u / np.linalg.norm(u)
        elif( b == 0 and a !=0 and c != 0):
            u = np.array([0, 0, -d/c]) - np.array([-d/a, 0, 0])
            u = u / np.linalg.norm(u)
        elif( c == 0 and a !=0 and b != 0):
            u = np.array([0, -d/b, 0]) - np.array([-d/a, 0, 0])
            u = u / np.linalg.norm(u)
        elif(a == 0 and b == 0):
            u = np.array([1, 0, 0])
        elif(a == 0 and c == 0):
            u = np.array([0, 0, 1])
        elif(b == 0 and c == 0):
            u = np.array([0, 0, 1])
        elif(a == 0 or b == 0 or c == 0):
            print("plane parameter error! a =", a, "b =", b, "c =", c, "d =", d)
        else:
            u = np.array([0, 0, -d/c]) - np.array([1, 1, (-d - a - b)/c])
            u = u / np.linalg.norm(u)
    else:
        u = ProjectVectorOnToPlane(prev_u, normal)

    v = np.cross(u, normal)
    v = normalize(v)
    prev_u = u

#    planeImg = np.zeros((halfDiagnalLength*2, halfDiagnalLength*2))
    
#     # filling the 2d Image, this part can be further optimized by using matrix
#     for x in range(-halfDiagnalLength, halfDiagnalLength):
#         for y in range(-halfDiagnalLength, halfDiagnalLength):
#             vectorElem1 = x*u
#             vectorElem2 = y*v
#             volumeCorFloat = vectorElem1 + vectorElem2 + centerPoint
#             volumeCor = np.array([int(round(volumeCorFloat[0])), int(round(volumeCorFloat[1])), int(round(volumeCorFloat[2]))])
#             #Check boundary
#             if(volumeCor[0] > 0 and volumeCor[0] < bounds[0] and volumeCor[1] > 0 and volumeCor[1] < bounds[1] and\
#               volumeCor[2] > 0 and volumeCor[2] < bounds[2]):
                
#                 volxelVal = data_volume[volumeCor[0]][volumeCor[1]][volumeCor[2]]
#                 planeImg[x + halfDiagnalLength][y + halfDiagnalLength] = volxelVal
    
    xyCorMatrix = [[x,y] for x in range(-resultRangeHalf, resultRangeHalf) \
                   for y in range(-resultRangeHalf, resultRangeHalf)]
    xyCorMatrix = np.array(xyCorMatrix)
    uvMat = np.array([u, v])
    dotMat = np.dot(xyCorMatrix, uvMat)
    dotMat += np.array(centerPoint)
    dotMat = np.round(dotMat).astype(int)    
    indexMat = np.transpose(dotMat)
    
    invalid = np.where((indexMat[0] <= 0) | (indexMat[0] >= bounds[0]) | \
                       (indexMat[1] <= 0) | (indexMat[1] >= bounds[1]) | \
                       (indexMat[2] <= 0) | (indexMat[2] >= bounds[2]))[0].tolist()
    
    indexMat[:, invalid] -= indexMat[:, invalid]
    
    rawImage = data_volume[indexMat[0], indexMat[1], indexMat[2]]
    planeImg = np.reshape(rawImage, (resultRangeHalf*2, resultRangeHalf*2))
    
    return (planeImg, prev_u)

In [ ]:
filePath = 'mri_image_2016/*.png'
data = ReadPointFromPNG(filePath, 2)

In [ ]:
plt.imshow(data[:, :, 100])

In [ ]:
plane = 0.05, -0.05, 1, -100
center = np.array([250, 250, 100])

In [ ]:
result = getTangentPlane (data, plane, center, 150)

In [ ]:
plt.imshow(result)

In [ ]:
# This function will invoke a pptk viewer to render the points 
# Param: data: an array of points coordinate. Each point's coordinate should has the format of [x, y, z]
#       pointSize: the size of the point to be rendered on the screen
# Return: none

def displayPoints(data, pointSize):
    v = pptk.viewer(data)
    v.set(point_size=pointSize)
    
def expand_centerline_coord(centerline_array):
    new_array = []
    for p in centerline_array:
        x = p[0] * 2
        y = p[1] 
        z = p[2] * 2
        point = [x,y,z]
        new_array.append(point)
        
    new_array = np.array(new_array)
    
    return new_array

In [ ]:
def VectorBetweenTwoPoints(p1, p2):
    return p2 - p1

def GetPlaneParam(p, n):
    a = n[0]
    b = n[1]
    c = n[2]
    d = - (n[0]*p[0] + n[1]*p[1] + n[2]*p[2])
    return (a,b,c,d)

def AssertSameVector(v1, v2):
    assert len(v1) == len(v2)
    for i in range (len(v1)):
        assert v1[i] == v2[i]

In [ ]:
def CollectPlanePointPair(centerline_points):
    plane_point_pairs = []
    
    i = 0
    for p in centerline_points:
        AssertSameVector(p, centerline_points[i])

        tangent = (0,0,0)
        if i - 1 >= 0 and i + 1 < len(centerline_points):
            tangent = VectorBetweenTwoPoints(centerline_points[i - 1], centerline_points[i + 1])
        elif i - 1 >= 0:
            tangent = VectorBetweenTwoPoints(centerline_points[i - 1], p)
        elif i + 1 < len(centerline_points):
            tangent = VectorBetweenTwoPoints(p, centerline_points[i+1])
        else:
            print ("seems that the list has only one or no element.")
            print ("the actual element num is:", len(centerline_points))
            break
            
        n = normalize(tangent)
        
        plane_param = GetPlaneParam(p,n)
        plane_point_pairs.append((plane_param, p))
        
        i += 1
        
    return plane_point_pairs

In [ ]:
def WriteProgress(toDeleteCount, progress_string):

    # sys.stdout.write("=>")

    for i in range(toDeleteCount):
        sys.stdout.write("\b \b")
        sys.stdout.flush()
    
    sys.stdout.write(progress_string)
    sys.stdout.flush()

def CollectResultImages(plane_point_pairs, save_image_sample: bool = False, ):
    results = []
    sys.stdout.write("Progress: ")
    sys.stdout.flush()
    os.system("mkdir ./Images")
    toDeleteCharCount = 0
    previous_u = [0,0,0]
    first_call = True
    
    for i in range(len(plane_point_pairs)):
        plane = plane_point_pairs[i][0]
        center = plane_point_pairs[i][1]
        (result, prev_u) = getSmoothTangentPlane (data, plane, center, previous_u, first_call, 150)
        previous_u = prev_u.tolist()
        results.append(result)
        filename = "Images/" + str(i) + ".png"
        plt.imsave(filename,result,cmap='gray', vmin=0, vmax=255)
        first_call = False
        
        if i % 10 == 0:
            progress_string = str(i) + "/" + str(len(plane_point_pairs))
            WriteProgress(toDeleteCharCount, progress_string)
            toDeleteCharCount = len(progress_string)
            
    print ("\n[ Done! We have", len(results), "images", "Saved to the folder called Images. ]")
    return results

In [ ]:
def BezierCurveCreator(p0, p1, p2, pointCount):
    pointList = []
    
    for i in range(pointCount):
        t = (i + 1) / float(pointCount)
        
        c0 = (1 - t) * (1 - t)
        c1 = 2 * (1 - t) * t
        c2 = t * t
        
        point = c0 * p0 + c1 * p1 + c2 * p2
        
        pointList.append(point)
    
    return pointList

def SmoothSuperSampling(ordered_points, segment_points_count: int = 200):
    smooth_curve_points = []
    
    num_points = len(ordered_points)
    # select the points that are will be gone through
    num_actual_points = (num_points - num_points % 2) / 2
    offset = num_points % 2
    num_actual_points = int(num_actual_points - 2 + offset)
    print ("num_actual_points:", num_actual_points)
    
    for i in range(num_actual_points):

        p0 = ordered_points[2 * i]
        p1 = ordered_points[2 * i + 1]
        p2 = ordered_points[2 * i + 2]
        
        if (2 * i - 1) >= 0:
            p0 = (ordered_points[2 * i - 1] + p1) / 2
        
        if (2 * i + 3) < num_points:
            p2 = (p1 + ordered_points[2 * i + 3]) / 2
        
        smooth_curve_points += BezierCurveCreator(p0, p1, p2, segment_points_count)
    
    return smooth_curve_points
    

In [ ]:
centerline_array = np.load("testInOrder.npy")
print(centerline_array.shape)

# centerline_array = expand_centerline_coord(centerline_array)
# displayPoints(centerline_array, 1.3)

In [ ]:
centerline_points = SmoothSuperSampling(centerline_array, 30)
plane_point_pairs = CollectPlanePointPair(centerline_points)

In [ ]:
len(centerline_points)

In [ ]:
result_images = CollectResultImages(plane_point_pairs, True)

In [ ]:
plt.imshow(result_images[1])

In [29]:
displayPoints(centerline_points, 0.05)

In [ ]:
def MakeMovieOutOfImage():
    image_folder = 'Images'
    video_name = 'video_complete.avi'
    
    fnames = []
    
    # reading and sorting file names
    for (dirpath, dirname, filename) in walk(image_folder):
        fnames.extend(filename)

        numbers = []
        fname_to_number = {}

        to_delete = []
        for fname in fnames:
            if(len(reg.findall(r'\d+', fname)) > 0):
                numbers.append(reg.findall(r'\d+', fname)[0])
                fname_to_number[fname] = reg.findall(r'\d+', fname)[0]
            else:
                print("Found weird file and ignored: " + str(fname))
                to_delete.append(fname)

        for ddd in to_delete:
            del fnames[fnames.index(ddd)]

        # fname_to_number = dict(zip(fnames,numbers))
        fnames.sort(key = lambda fname : int(fname_to_number[fname]))
        
    image_name_list = fnames
    images = [img for img in image_name_list if img.endswith(".png")]
    frame = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, layers = frame.shape

    video = cv2.VideoWriter(video_name, 0, 30, (width,height))

    for image in images:
        video.write(cv2.imread(os.path.join(image_folder, image)))

    cv2.destroyAllWindows()
    video.release()

In [ ]:
MakeMovieOutOfImage()